C-7.24 Give a complete implementation of the stack ADT using a singly linked list that includes a header sentinel.

In [ ]:
class Node:
    def __init__(self, data=None):
        self.data = data
        self.next = None

class Stack:
    def __init__(self):
        self.header = Node()
        self.header.next = None
        self.size = 0

    def is_empty(self):
        return self.size == 0

    def add(self, data):
        new_node = Node(data)
        new_node.next = self.header.next
        self.header.next = new_node
        self.size += 1

    def subtract(self):
        if self.is_empty():
            raise IndexError("pop from empty stack")
        top_node = self.header.next
        self.header.next = top_node.next
        self.size -= 1
        return top_node.data

    def peek(self):
        if self.is_empty():
            raise IndexError("peek from empty stack")
        return self.header.next.data

    def __len__(self):
        return self.size

    def __str__(self):
        current = self.header.next
        s = []
        while current is not None:
            s.append(current.data)
            current = current.next
        return "Stack(top -> bottom): " + " -> ".join(map(str, s))


In [ ]:
stack = Stack()
stack.add(10)
stack.add(20)
stack.add(30)
print(stack)
print(stack.subtract())
print(stack.peek())
print(stack.is_empty())
print(len(stack))
stack.subtract()
stack.subtract()
print(stack.is_empty())

Stack(top -> bottom): 30 -> 20 -> 10
30
20
False
2
True


P-7.44 Write a simple text editor that stores and displays a string of characters using the positional list ADT, together with a cursor object that highlights a position in this string. A simple interface is to print the string and then to use a second line of output to underline the position of the cursor. Your editor should support the following operations:

 • left: Move cursor left one character (do nothing if at beginning).

 • right: Move cursor right one character (do nothing if at end).

 • insert c: Insert the character c just after the cursor.

 • delete: Delete the character just after the cursor (do nothing at end).






In [1]:
class Node:
    def __init__(self, element=None, prev=None, next=None):
        self.element = element
        self.next = next
        self.prev = prev


class PositionalList:

    class Position:
        def __init__(self, container, node):
            self._container = container
            self._node = node

        def element(self):
            return self._node.element

        def __eq__(self, other):
            return type(other) is type(self) and other._node is self._node

        def __ne__(self, other):
            return not (self == other)

    def __init__(self):
        self._header = Node()
        self._trailer = Node()
        self._header.next = self._trailer
        self._trailer.prev = self._header
        self._size = 0

    def _make_position(self, node):
        if node is self._header or node is self._trailer:
            return None
        else:
            return self.Position(self, node)

    def _validate(self, p):
        if not isinstance(p, self.Position):
            raise TypeError("p must be proper Position type")
        if p._container is not self:
            raise ValueError("p does not belong to this container")
        if p._node._next is None:
            raise ValueError("p is no longer valid")
        return p._node

    def first(self):
        return self._make_position(self._header.next)

    def last(self):
        return self._make_position(self._trailer.prev)

    def __len__(self):
        return self._size

    def is_empty(self):
        return self._size == 0

    def _insert_between(self, element, predecessor, successor):
        new_node = Node(element, predecessor, successor)
        predecessor.next = new_node
        successor.prev = new_node
        self._size += 1
        return self.Position(self, new_node)

    def add_first(self, element):
        return self._insert_between(element, self._header, self._header.next)

    def add_last(self, element):
        return self._insert_between(element, self._trailer.prev, self._trailer)

    def add_after(self, p, e):
        original = self._validate(p)
        return self._insert_between(e, original, original.next)

    def delete(self, p):
        node = p._node
        predecessor = node.prev
        successor = node.next
        predecessor.next = successor
        successor.prev = predecessor
        self._size -= 1
        element = node.element
        node.prev = node.next = node.element = None
        return element

class TextEditor:
    def __init__(self):
        self.text = PositionalList()
        self.cursor_index = 0

    def display(self):
        current = self.text._header.next
        index = 0
        while current is not self.text._trailer:
            if index == self.cursor_index:
                print('|', end='')
            print(current.element, end='')
            current = current.next
            index += 1
        if self.cursor_index == len(self.text):
            print('|', end='')
        print()

    def left(self):
        self._move_cursor_left()

    def right(self):
        self._move_cursor_right()

    def _get_position_at_index(self, index):
        current = self.text._header.next
        for _ in range(index):
            current = current.next
        return self.text._make_position(current)

    def _move_cursor_left(self):
        if self.cursor_index > 0:
            self.cursor_index -= 1
        self.display()

    def _move_cursor_right(self):
        if self.cursor_index < len(self.text):
            self.cursor_index += 1
        self.display()

    def insert(self, char):
        if self.cursor_index == len(self.text):
            self.text.add_last(char)
        else:
            current_position = self._get_position_at_index(self.cursor_index)
            self.text.add_after(current_position, char)
        self._move_cursor_right()

    def delete(self):
        if self.cursor_index < len(self.text):
            current_position = self._get_position_at_index(self.cursor_index)
            self.text.delete(current_position)
        self.display()


def main():
    text_editor = TextEditor()
    print("Text Editor")
    print("Commands:")
    print("  left       - Move cursor left one character (do nothing if at beginning)")
    print("  right      - Move cursor right one character (do nothing if at end)")
    print("  insert c   - Insert character 'c' just after the cursor")
    print("  delete     - Delete the character just after the cursor (do nothing at end)")
    print("  end        - Terminate program")

    while True:
        command = input("\nEnter command: ").strip().split()
        if not command:
            continue

        action = command[0].lower()

        if action == "left":
            text_editor.left()
        elif action == "right":
            text_editor.right()
        elif action == "insert":
            if len(command) == 2 and len(command[1]) == 1:
                text_editor.insert(command[1])
            else:
                print("Invalid insert command. Use: insert c (where c is a single character)")
        elif action == "delete":
            text_editor.delete()
        elif action == "end":
            print("Terminating program. Thank you.")
            break
        else:
            print("Invalid command. Try again.")

main()

Text Editor
Commands:
  left       - Move cursor left one character (do nothing if at beginning)
  right      - Move cursor right one character (do nothing if at end)
  insert c   - Insert character 'c' just after the cursor
  delete     - Delete the character just after the cursor (do nothing at end)
  end        - Terminate program

Enter command: insert h
h|

Enter command: insert i
hi|

Enter command: insert 
Invalid insert command. Use: insert c (where c is a single character)

Enter command: left
h|i

Enter command: delete
h|

Enter command: left
|h

Enter command: right
h|

Enter command: end
Terminating program. Thank you.
